# Imports

In [1]:
%%time
import pytorch_lightning as pl
import torch as th

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, GPUStatsMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics.functional.classification import accuracy
from pytorch_lightning import seed_everything
import torch.nn.functional as F
import torchvision
import timm
from timm import create_model
from torchvision import transforms
import os
import pandas as pd

# data module
from src.dataset import DataModule

# model
from src.model import Model

# config file
from src.config import Config

CPU times: user 1.25 s, sys: 325 ms, total: 1.57 s
Wall time: 1.28 s


# Data module and setup

In [2]:
_ = seed_everything(seed =  Config.seed_val)

Global seed set to 2021


In [3]:
config_dict = Config.__dict__.items()
config_dict = dict([item for item in config_dict if '__' not in item[0]])
config_dict

{'data_dir': '/home/zeusdric/Dric/Zindi2020/Coding-Room/projects/cassava-leaf-disease/data/input',
 'models_dir': '/home/zeusdric/Dric/Zindi2020/Coding-Room/projects/cassava-leaf-disease/models',
 'logs_dir': '/home/zeusdric/Dric/Zindi2020/Coding-Room/projects/cassava-leaf-disease/logs',
 'train_data_dir': '/home/zeusdric/Dric/Zindi2020/Coding-Room/projects/cassava-leaf-disease/data/input/train_images',
 'test_data_dir': '/home/zeusdric/Dric/Zindi2020/Coding-Room/projects/cassava-leaf-disease/data/input/test_images',
 'num_epochs': 5,
 'lr': 0.01,
 'resize': 400,
 'img_h': 350,
 'img_w': 350,
 'weight_decay': 0.01,
 'eps': 1e-08,
 'train_batch_size': 32,
 'test_batch_size': 32,
 'base_model': 'resnet50',
 'seed_val': 2021}

In [4]:
PATH = Config.data_dir
PATH

'/home/zeusdric/Dric/Zindi2020/Coding-Room/projects/cassava-leaf-disease/data/input'

In [5]:
%%time
train_df = pd.read_csv(os.path.join(Config.data_dir, 'train.csv'))
data_transform = {
    'train': transforms.Compose([
        transforms.Resize(size=(Config.resize, Config.resize)),
        transforms.RandomHorizontalFlip(p=.7),
        transforms.RandomVerticalFlip(p=.3),
        transforms.RandomRotation(degrees=25),
        transforms.CenterCrop(size=(Config.img_h, Config.img_w)),
        transforms.ColorJitter(brightness=(0.4, 1), contrast=.2, saturation=0, hue=0),
        transforms.GaussianBlur(kernel_size=3)
    ]),
    
    'validation':transforms.Compose([
        transforms.Resize(size=(Config.resize, Config.resize)),
        transforms.RandomRotation(degrees=25),
        transforms.CenterCrop(size=(Config.img_h, Config.img_w)),
        transforms.ColorJitter(brightness=(0.45, 1), 
                               contrast=.1, 
                               saturation=.1, 
                               hue=0.1),
        transforms.GaussianBlur(kernel_size=3)
    ]), 
    
    'test':transforms.Compose([
        transforms.Resize(size=(Config.img_h, Config.img_w)),
        transforms.RandomRotation(degrees=25),
    ])
    
}
dm = DataModule(config=Config, 
                 train_data_dir=Config.train_data_dir, 
                 test_data_dir=Config.test_data_dir, 
                 train_df=train_df,
                 data_transform=data_transform,
                 validation_split=.11,
                 train_frac = 1)
dm.setup()

[INFO] Training on 19043
[INFO] Validating on 2354
CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 13.8 ms


In [6]:
%%time
model = Model(config=config_dict)

CPU times: user 519 ms, sys: 97.5 ms, total: 617 ms
Wall time: 531 ms


In [7]:
%%time
ckpt_cb = ModelCheckpoint(
    monitor='val_acc', 
    mode='max', 
    dirpath=Config.models_dir, 
    filename='leaf_disease_classifier-{val_acc:.5f}-{val_loss:.5f}'
)

gpu_stats = GPUStatsMonitor(
    memory_utilization=True, 
    gpu_utilization=True, 
    fan_speed=True, 
    temperature=True
)
es = EarlyStopping(
    monitor='val_acc', 
    patience=4, 
    mode='max'
)

Logger = TensorBoardLogger(
    save_dir=Config.logs_dir, 
    name='cassava_leaf_disease'
)

Callbacks = [es, ckpt_cb, gpu_stats]

trainer = pl.Trainer(
    gpus=-1, 
    max_epochs=Config.num_epochs, 
    precision=16,
    callbacks=Callbacks,
    logger=Logger,
    #fast_dev_run=True
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


CPU times: user 34.5 ms, sys: 11.6 ms, total: 46.1 ms
Wall time: 19.6 ms


# Training phase

In [8]:
%%time
trainer.fit(model=model, datamodule=dm)


  | Name       | Type    | Params
---------------------------------------
0 | encoder    | ResNet  | 25.6 M
1 | classifier | Linear  | 5.0 K 
2 | dropout    | Dropout | 0     
---------------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/zeusdric/anaconda3/envs/deeplearning/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


MisconfigurationException: ModelCheckpoint(monitor='val_acc') not found in the returned metrics: ['train_acc_step']. HINT: Did you call self.log('val_acc', tensor) in the LightningModule?

In [9]:
%load_ext tensorboard

In [10]:
%tensorboard --logdir ../logs

Reusing TensorBoard on port 6006 (pid 629950), started 0:50:02 ago. (Use '!kill 629950' to kill it.)